In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
from PSG_PDF import PDF, report_data
from pylab import rcParams
from tqdm import tqdm

rcParams['figure.figsize'] = 12,4

import warnings
warnings.filterwarnings('ignore')

def transform_list(nums):
    # 숫자와 원래 인덱스를 함께 저장
    indexed_nums = list(enumerate(nums))
    # 숫자 기준으로 정렬
    indexed_nums.sort(key=lambda x: x[1])
    
    # 변환된 순서를 저장할 리스트 초기화
    transformed = [0] * len(nums)
    # 첫 번째 숫자에 대한 순서는 항상 1
    current_rank = 1
    
    for i in range(len(nums)):
        if i > 0 and indexed_nums[i][1] != indexed_nums[i-1][1]:
            current_rank += 1  # 숫자가 이전 숫자와 다르면 순위 증가
        transformed[indexed_nums[i][0]] = current_rank  # 원래 인덱스에 순위 저장
    
    return transformed

In [7]:
day = '20240302'
store = f'processed file/{day}/report'

kine_list = [i.replace('\\','/') for i in sorted(glob(f'processed file/{day}/kine/*csv'))][0:1]
force_list = [i.replace('\\','/') for i in sorted(glob(f'processed file/{day}/force/*csv'))][0:1]
comment = f'comment_file/{day}'

kine_name = [f"{os.path.basename(i).split('.')[0]}" for i in kine_list]
force_name = [f"{os.path.basename(i).split('.')[0]}" for i in force_list]

comment

'comment_file/20240302'

In [8]:
kine_data = {}
for k, n in zip(kine_list, kine_name):
    kine_data[n] = k
    
force_data = {}
for f, n in zip(force_list, force_name):
    force_data[n] = f

In [9]:
kine_data, force_data

({'songseokhyun_0001_L_91_185_OH_88_B': 'processed file/20240302/kine/songseokhyun_0001_L_91_185_OH_88_B.csv'},
 {'songseokhyun_0001_L_91_185_OH_88_B': 'processed file/20240302/force/songseokhyun_0001_L_91_185_OH_88_B.csv'})

In [10]:
class PDF(PDF):
    def header(self):
        # Logo
        self.set_font('helvetica', 'I', 8)
        pdf.set_text_color(150,0,0)
        # pdf.cell(0, -10, text = "Deifining Parameters : Click [https://github.com/parkdragonstone/kookmin-report-explain]", ln = True, align = '', link = "https://github.com/parkdragonstone/kookmin-report-explain")
        
    # Page footer
    def footer(self):
        # Set position of the footer
        self.set_y(-15)
        # set font
        self.set_font('helvetica', 'I', 8)
        # Page number
        self.cell(0, 10, f"Page {self.page_no()}" + "/" +f"{{nb}}", align = "C")
        # Logo
        self.image(f'imagefile/logo.png',
                160, 280, 50)
    
    # Adding chapter title to start of each chapter
    def chapter_title(self, ch_num, ch_title):
        # set font
        self.set_font('helvetica', '', 12)
        # background color
        self.set_fill_color(200, 220, 255)
        # Chapter title
        chapter_title = f'Chapter {ch_num} : {ch_title}'
        self.cell(0, 5, chapter_title, ln=1, fill=1)
        # line break
        self.ln()

    # Chapter content
    def chapter_body(self):
        # end each chapter
        self.set_font('helvetica', 'I', 12)
        self.cell(0, 5, 'END OF CHAPTER')

    def print_chapter(self, ch_num, ch_title):
        self.add_page()
        self.chapter_title(ch_num, ch_title)
        self.chapter_body()
        
ks_cols = {
    'PELVIS_ANGLUAR_VELOCITY_Z'        : ['PELVIS'   , 'r'],
    'TORSO_ANGLUAR_VELOCITY_Z'         : ['TORSO'    , 'g'],
    'LEAD_ELBOW_ANGULAR_VELOCITY_X'    : ['ELBOW'    , 'b'],
    'LEAD_SHOULDER_ANGULAR_VELOCITY_Z' : ['SHOULDER' , 'y'],
}

ang_cols = {
    'TORSO_PELVIS_ANGLE_Z'            : 'Hip-Shoulder Separation',
    'LEAD_ELBOW_ANGLE_X'              : 'ELBOW FLEXION',
    'LEAD_SHOULDER_ANGLE_Z'           : 'SHOULDER EXTERNAL ROTATION',          
    'LEAD_SHOULDER_ANGLE_X'           : 'SHOULDER HORIZONTAL ABDUCTION',
    'LEAD_KNEE_ANGULAR_VELOCITY_X'    : 'LEAD LEG KNEE EXTENSION ANGULAR VELOCITY',
    'LEAD_SHOULDER_ANGLE_Y'           : 'SHOULDER ABDUCTION', 
    'LEAD_KNEE_ANGLE_X'               : 'LEAD LEG KNEE FLEXION',
    'TORSO_ANGLE_X'                   : 'TRUNK FORWARD TILT',
    'TORSO_ANGLE_Y'                   : 'TRUNK LATERAL TILT',
}

ap_cols = {
    'REAR_FORCE_Y' : ['Trail Leg' , 'b'],
    'LEAD_FORCE_Y' : ['Stride Leg', 'r'],
}

vt_cols = {
    'REAR_FORCE_Z' : ['Trail Leg' , 'b'],
    'LEAD_FORCE_Z' : ['Stride Leg', 'r']
}

for name in tqdm(kine_data):
    kine = kine_data[name]
    force = force_data[name]

    k_df = pd.read_csv(kine)
    f_df = pd.read_csv(force)

    k_sr = 180
    k_kh_time  = k_df['kh_time'][0]
    k_kh_time1 = k_df['kh_time'][0] - k_kh_time
    k_fc_time  = k_df['fc_time'][0] - k_kh_time
    k_fp_time  = k_df['fp_time'][0] - k_kh_time
    k_mer_time = k_df['mer_time'][0] - k_kh_time
    k_br_time  = k_df['br_time'][0] - k_kh_time
    stride_length = round(k_df['stride_length'][0])
    
    f_sr = 1080
    f_kh_time  = f_df['kh_time'][0] 
    f_kh_time1 = f_df['kh_time'][0]  - f_kh_time
    f_fc_time  = f_df['fc_time'][0]  - f_kh_time
    f_fp_time  = f_df['fp_time'][0]  - f_kh_time
    f_mer_time = f_df['mer_time'][0] - f_kh_time
    f_br_time  = f_df['br_time'][0]  - f_kh_time

    k_df = k_df.iloc[k_kh_time:int(k_br_time + k_kh_time + (k_sr * 0.2)),:].reset_index(drop=True)
    f_df = f_df.iloc[f_kh_time:int(f_br_time + f_kh_time + (f_sr * 0.2)),:].reset_index(drop=True)

    f_lead_peak_z_time = f_df['lead_peak_z'][0] - f_kh_time
    f_rear_peak_z_time = np.where(f_df['REAR_FORCE_Z'] == f_df['REAR_FORCE_Z'].max())[0][0]
    force_peak_time = round((f_lead_peak_z_time - f_rear_peak_z_time) / 1080 , 4)
    
    f_rear_peak_y_time = np.where(f_df['REAR_FORCE_Y'] == f_df['REAR_FORCE_Y'].max())[0][0]
    f_lead_min_y_time  = f_df['lead_valley_y'][0] - f_kh_time

    k_df.drop(['kh_time','fc_time','fp_time','mer_time','br_time','mir_time'], axis=1, inplace=True)
    f_df.drop(['kh_time','fc_time','fp_time','mer_time','br_time','mir_time'], axis=1, inplace=True)

    k_len = len(k_df)
    k_time = np.arange(0,k_len/k_sr, 1/k_sr)

    f_len = len(f_df)
    f_time = np.arange(0,f_len/f_sr, 1/f_sr)

    k_fc_br = k_br_time - k_fp_time
    f_fc_br = f_br_time - f_fp_time

    y   = report_data.grf(f_df, ap_cols, f_time, f_kh_time1, f_fc_time, f_mer_time, f_br_time,f_rear_peak_z_time,f_lead_peak_z_time,f_rear_peak_y_time,f_lead_min_y_time, axis='ap')
    z   = report_data.grf(f_df, vt_cols, f_time, f_kh_time1, f_fc_time, f_mer_time, f_br_time,f_rear_peak_z_time,f_lead_peak_z_time,f_rear_peak_y_time,f_lead_min_y_time, axis='vt')
    ang = report_data.one_angle(k_df, ang_cols, k_time, k_kh_time1, k_fc_time, k_mer_time, k_br_time)
    ks  = report_data.kinematic_sequence(k_df, ks_cols, k_time, k_fc_time, k_mer_time, k_br_time)

    with open(f"{comment}/{name.split('_')[0]}_{name.split('_')[1]}.txt", "rb") as fh:
        comment_txt = fh.read().decode('utf-8')


    pdf = PDF("P", "mm","A4")
    pdf.set_auto_page_break(auto = True, margin = 15)  
    pdf.add_font(family = 'kor', style = 'B', fname =  'NanumGothicCoding-Bold.ttf', uni=True)
    pdf.add_font(family = 'kor', style = '', fname =  'NanumGothicCoding.ttf', uni=True)  
    # main page
    pdf.add_page()
    pdf.set_font('helvetica', 'I', 8)
    pdf.set_text_color(150,0,0)
    pdf.cell(120, 0, text = "Deifining Parameters : Click [https://github.com/parkdragonstone/kookmin-report-explain]", ln = False, align = '', link = "https://github.com/parkdragonstone/kookmin-report-explain")
    pdf.set_text_color(0,0,0)
    pdf.set_font('helvetica', 'BU', 20)
    title = 'Kookmin University Pitching Report'
    pdf.ln(3)
    title_w = pdf.get_string_width(title) + 5
    doc_w = pdf.w
    pdf.set_x((doc_w - title_w)/2)
    pdf.cell(135, 15, title, align = 'C', ln = True)
    pdf.set_fill_color(200, 220, 255)

    # ===========================================================================================================
    # chapter
    player, trial, side, weight, height, type, ballspeed, _= name.split('_')
    ballspeed = 1.6 * int(ballspeed)
    if side == 'R':
        side = 'Right'
    elif side == 'L':
        side = 'Left'

    if type == 'OH':
        type = 'Overhead'
    elif type == 'UD':
        type = 'Under'
    elif type == 'TQ':
        type = 'ThreeQuater'
    elif type == 'SD':
        type = 'Side'

    ch1_link = pdf.add_link()
    ch2_link = pdf.add_link()
    ch3_link = pdf.add_link()
    ch4_link = pdf.add_link()
    ch5_link = pdf.add_link()
    ch6_link = pdf.add_link()
    ch7_link = pdf.add_link()
    ch8_link = pdf.add_link()

    pdf.set_link(link = ch1_link, page=1)
    pdf.set_link(link = ch2_link, page=2)
    pdf.set_link(link = ch3_link, page=3)
    pdf.set_link(link = ch4_link, page=4)
    pdf.set_link(link = ch5_link, page=5)
    pdf.set_link(link = ch6_link, page=6)
    pdf.set_link(link = ch7_link, page=7)
    # pdf.set_link(link = ch8_link, page=8)
    
    
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.set_font('helvetica', 'B', 8)
    pdf.cell(0, 2,"",ln=1,align = '')
    pdf.cell(0, 0, f"Name : {player}    Date : {day}   Hand : {side}   Type : {type}    Ball Velocity : {ballspeed} km/h   Weight : {weight}kg    Height : {height} cm", align = 'C', ln = True)
    pdf.cell(0, 5,"",ln=1,align = '')
    pdf.image(f'imagefile/analysis.png',x= 30,w=140,h=50)

    # Set font
    pdf.set_font('helvetica', 'B', 11)

    # Function to add a section heading
    def add_section_heading(text, link,ln, x=None, y=None):
        pdf.set_fill_color(200, 220, 255)  # Blue fill
        pdf.set_text_color(0, 0, 0)  # Black text
        if x is not None:
            pdf.set_x(x)
        if y is not None:
            pdf.set_y(y)
        pdf.cell(90, 6, text, ln=ln, align='C', fill=True, link=link)

    # Function to add an item
    def add_item(text, link, ln, x=None):
        pdf.set_text_color(0, 0, 0)  # Black text
        if x is not None:
            pdf.set_x(x)
        pdf.cell(90, 4, text, ln=ln, align='', link=link)

    pdf.ln(4)

    # Add first row of section headings
    add_section_heading('PITCHING EFFICIENCY', ch1_link, False)
    add_section_heading('STRIDE', ch2_link, 1, 110)
    pdf.cell(0, 2,"",ln=1,align = 'C')
    
    # Add items for the first two sections
    add_item('1. KINEMATIC SEQUENCE & STRIDE LENGTH', ch1_link, False )
    add_item('1. HIP/SHOULDER SEPARATION', ch2_link, 1, 110)
    pdf.ln(4)
    add_item('2. ELBOW FLEXION', ch2_link, 1, 110)
    pdf.ln(4)
    add_item('3. GROUND REACTION FORCE(TRAIL LEG)', ch2_link, 1, 110)
    pdf.ln(4)
    # Add second row of section headings
    add_section_heading('ARM COCKING', ch3_link, False)
    add_section_heading('ARM ACCELERATION', ch5_link, 1, 110)
    pdf.ln(4)
    # Add items for the third and fourth sections
    add_item('1. SHOULDER EXTERNAL ROTATION', ch3_link, False)
    add_item('1. SHOULDER ABDUCTION', ch5_link, 1, 110)
    pdf.ln(4)
    add_item('2. SHOULDER HORIZONTAL ABDUCTION', ch4_link, False)
    add_item('2. TRUNK FORWARD TILT', ch6_link, 1, 110)
    pdf.ln(4)
    add_item('3. LEAD LEG KNEE FLEXION', ch4_link, False)
    add_item('3. TRUNK LATERAL TILT', ch6_link, 1, 110)
    pdf.ln(4)
    add_item('4. GROUND REACTION FORCE(LEAD LEG)', ch4_link, False)
    add_item('4. LEAD LEG KNEE EXTENSION', ch6_link, 1, 110)
    add_item('  ANGULAR VELOCITY', ch6_link, 1, 110)

    pdf.set_font('helvetica', 'B', 11)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.cell(0, 7, 'COMMENT', ln = 1, align = 'C',link = ch7_link, fill = True)
    
    # ===========================================================================================================
    
    # kinematic sequence

    peak_pel = round(ks['peak']['PELVIS_ANGLUAR_VELOCITY_Z']); time_pel = ks['time']['PELVIS_ANGLUAR_VELOCITY_Z']
    peak_tor = round(ks['peak']['TORSO_ANGLUAR_VELOCITY_Z']);time_tor = ks['time']['TORSO_ANGLUAR_VELOCITY_Z']
    peak_elb = round(ks['peak']['LEAD_ELBOW_ANGULAR_VELOCITY_X']);time_elb = ks['time']['LEAD_ELBOW_ANGULAR_VELOCITY_X']
    peak_sho = round(ks['peak']['LEAD_SHOULDER_ANGULAR_VELOCITY_Z']);time_sho = ks['time']['LEAD_SHOULDER_ANGULAR_VELOCITY_Z']
    #########################################################
    total_time = k_br_time - k_fc_time
    #########################################################
    pel_time = round(100 * (time_pel - k_fc_time) / total_time)
    tor_time = round(100 * (time_tor - k_fc_time) / total_time)
    elb_time = round(100 * (time_elb - k_fc_time) / total_time)
    sho_time = round(100 * (time_sho - k_fc_time) / total_time)
    
    tor_gain = round(peak_tor / peak_pel,2)
    upper_gain = round(peak_elb / peak_tor,2)
    fore_gain = round(peak_sho / peak_elb,2)

    #pdf.set_link(ch1_link, page = 2)
    pdf.set_y(188)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, '1. KINEMATIC SEQUENCE & STRIDE LENGTH', ln = True, link = ch1_link)
    pdf.set_font('helvetica', 'B', 7)
    pdf.set_x(105)
    pdf.set_text_color(150,0,0)
    pdf.cell(0, 0, 'Click [Go to Comment]', ln = True, link = ch7_link)
    pdf.set_text_color(0,0,0)
    pdf.set_y(188)
    pdf.set_font('helvetica', 'I', 10)
    #pdf.cell(0, 0, 'The order and velocity of the pelvis, trunk and arm rotation')
    pdf.image(f'figure/kinematic.png',x=65,y=193,w=135,h=45)
    if side == 'Right':
        pdf.image(f'imagefile/R/kinematic.png',x=10,y=190,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/kinematic.png',x=10,y=190,w=50,h=50)

    sq_time = [pel_time, tor_time, elb_time, sho_time]
    expected_order = transform_list(sq_time)

    data_as_dict = {"Segment"    : [   "Pelvic [°/s]",     "Torso [°/s]",      "Elbow [°/s]",  "Shoulder [°/s]"],
                    "Pro"        : [      "649 ~ 840",      "987 ~ 1174",      "2211 ~ 2710",     "4331 ~ 4884"],
                    "Peak"       : [         peak_pel,          peak_tor,           peak_elb,          peak_sho],
                    "Timing"     : [  f"{pel_time} %",   f"{tor_time} %",    f"{elb_time} %",   f"{sho_time} %"],
                    "Sequence"   : [expected_order[0], expected_order[1],  expected_order[2], expected_order[3]],
                    "Speed Gain" : [''               ,          tor_gain,         upper_gain,         fore_gain]}
    
    pdf.set_y(244)

    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [33,33,33,33,33,33],
                    x_start='C')
    
    pdf.set_text_color(150,0,0)
    pdf.set_y(237)
    pdf.set_font('helvetica', 'B', 8)
    pdf.cell(5, 7, f'Stride Length : {stride_length} % (% Height)', ln = 1, align = '')
    pdf.set_text_color(0,0,0)

    # ===========================================================================================================
    
    pdf.add_page()

    pdf.set_y(9)
    pdf.set_font('helvetica', 'B', 15)
    pdf.cell(190, 7, 'STRIDE', align = 'C', ln = True, fill = True)

    # HIP/SHOULDER SEPARATION
    
    #pdf.set_link(ch2_link, page = 2)
    pdf.set_y(20)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, '2. HIP/SHOULDER SEPARATION', ln = True, link = ch1_link)
    pdf.set_font('helvetica', 'B', 7)
    pdf.set_x(80)
    pdf.set_text_color(150,0,0)
    pdf.cell(0, 0, 'Click [Go to Comment]', ln = True, link = ch7_link)
    pdf.set_text_color(0,0,0)
    pdf.set_y(26)
    pdf.set_font('helvetica', 'I', 10)
    #pdf.cell(0, 0, 'The angle between the trunk and the pelvis with respect to the ground')
    pdf.image(f'figure/Hip-Shoulder Separation.png',x=70,y=23,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/Hip-Shoulder Separation.png',x=10,y=25,w=50,h=40)
    elif side == 'Left':
        pdf.image(f'imagefile/L/Hip-Shoulder Separation.png',x=10,y=25,w=50,h=40)
    
    data_as_dict = {
                    ""       : ["Hip Shoulder Separation [°]"],
                    "FC"     : [round(ang['fc_time']['TORSO_PELVIS_ANGLE_Z'])],
                    "MER"    : [round(ang['mer_time']['TORSO_PELVIS_ANGLE_Z'])],
                    "BR"     : [round(ang['br_time']['TORSO_PELVIS_ANGLE_Z'])],
                    "Min"    : [round(ang['min']['TORSO_PELVIS_ANGLE_Z'])],
                    "Timing" : [round(100 * (ang['min_frame']['TORSO_PELVIS_ANGLE_Z'] - k_fc_time) / (k_br_time - k_fc_time))]
                    }

    pdf.set_y(73)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [48,30,30,30,30,30],
                    x_start='C')
    
    report_data.create_bullet_chart('Hip Shoulder Separation (FC)', "FC", round(ang['fc_time']['TORSO_PELVIS_ANGLE_Z']), -60, -39, -25, 0, unit='°')
    pdf.image(f'figure/bar//Hip Shoulder Separation (FC)_bar.png', -15, 88, 235,)

    # ===========================================================================================================
    
    # ELBOW FLEXION

    #pdf.set_link(ch3_link, page = 2)
    pdf.set_y(107)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, '3. ELBOW FLEXION', ln = True, link = ch1_link)
    pdf.set_font('helvetica', 'B', 7)
    pdf.set_x(52)
    pdf.set_text_color(150,0,0)
    pdf.cell(0, 0, 'Click [Go to Comment]', ln = True, link = ch7_link)
    pdf.set_text_color(0,0,0)
    pdf.set_y(108)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'The sagittal plane rotation of the forearm relative to the upper arm')
    pdf.image(f'figure/ELBOW FLEXION.png',x=70,y=108,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/ELBOW FLEXION.png',x=10,y=110,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/ELBOW FLEXION.png',x=10,y=110,w=50,h=50)

    data_as_dict = {
                    ""       : ["Elbow Flexion [°]"],
                    "FC"     : [round(ang['fc_time']['LEAD_ELBOW_ANGLE_X'])],
                    "MER"    : [round(ang['mer_time']['LEAD_ELBOW_ANGLE_X'])],
                    "BR"     : [round(ang['br_time']['LEAD_ELBOW_ANGLE_X'])],
                    "Max"    : [round(ang['max']['LEAD_ELBOW_ANGLE_X'])],
                    "Timing" : [round(100 * (ang['max_frame']['LEAD_ELBOW_ANGLE_X'] - k_fc_time) / (k_br_time - k_fc_time))]
                    }
    
    pdf.set_y(159)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [48,30,30,30,30,30],
                    x_start='C')
    
    report_data.create_bullet_chart('Elbow Flexion (FC)', "FC", round(ang['fc_time']['LEAD_ELBOW_ANGLE_X']), 50, 76, 115, 140, unit='°')
    pdf.image(f'figure/bar//Elbow Flexion (FC)_bar.png', -15, 174, 235,)

    report_data.create_bullet_chart('Elbow Flexion (MER)', "MER", round(ang['mer_time']['LEAD_ELBOW_ANGLE_X']), 50, 82, 98, 130, unit='°')
    pdf.image(f'figure/bar//Elbow Flexion (MER)_bar.png', -15, 186, 235,)

    # ===========================================================================================================

    # GRF Y

    #pdf.set_link(ch11_link, page = 2)
    pdf.set_y(208)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, '4-1. TRAIL LEG GROUND REACTION FORCE (AP axis)', ln = True, link = ch1_link)
    pdf.set_font('helvetica', 'B', 7)
    pdf.set_x(122)
    pdf.set_text_color(150,0,0)
    pdf.cell(0, 0, 'Click [Go to Comment]', ln = True, link = ch7_link)
    pdf.set_text_color(0,0,0)
    pdf.set_y(208)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/grf_ap.png',x=70,y=213,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/GRF_Y.png',x=17,y=213,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/GRF_Y.png',x=17,y=213,w=50,h=50)

    data_as_dict = {
                    ""       : ["Trail Leg [% BW]"],
                    "KH"     : [round(y['kh_time']['REAR_FORCE_Y'])],
                    "FC"     : [round(y['fc_time']['REAR_FORCE_Y'])],
                    "Max"    : [round(y['max']['REAR_FORCE_Y'])],
                    "Timing" : [round(100 * (y['max_frame']['REAR_FORCE_Y'] - f_fc_time) / (f_br_time - f_fc_time))]
                    }
    
    pdf.set_y(264)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    #report_data.create_bullet_chart('GRF_YT', "Trail Leg MAX", round(y['max']['REAR_FORCE_Y']), 50, 93, 106, 130, unit='')
    #pdf.image(f'figure/bar/GRF_YT_bar.png', -15, 272, 235,)

    # ===========================================================================================================

    pdf.add_page()

    # GRF Z

    #pdf.set_link(ch12_link, page = 3)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, '4-2. TRAIL LEG  GROUND REACTION FORCE (Vertical)', ln = True, link = ch1_link)
    pdf.set_font('helvetica', 'B', 7)
    pdf.set_x(125)
    pdf.set_text_color(150,0,0)
    pdf.cell(0, 0, 'Click [Go to Comment]', ln = True, link = ch7_link)
    pdf.set_text_color(0,0,0)
    pdf.set_y(20)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/grf_vt.png',x=70,y=20,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/GRF_Z.png',x=16,y=20,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/GRF_Z.png',x=16,y=20,w=50,h=50)

    data_as_dict = {
                    ""       : ["Trail Leg [% BW]"],
                    "KH"     : [round(z['kh_time']['REAR_FORCE_Z'])],
                    "FC"     : [round(z['fc_time']['REAR_FORCE_Z'])],
                    "Max"    : [round(z['max']['REAR_FORCE_Z'])],
                    "Timing" : [round(100 * (z['max_frame']['REAR_FORCE_Z'] - f_fc_time) / (f_br_time - f_fc_time))]
                    }
    
    pdf.set_y(72)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    # pdf.set_y(12)
    # pdf.set_x(125)
    # pdf.set_text_color(150,0,0)
    # pdf.set_font('helvetica', 'B', 9)
    # pdf.cell(5, 7, f'Blue Vertical Line ~ Red Vertical Line Time : {force_peak_time} s', ln = 1, align = '')
    # pdf.set_text_color(0,0,0)

    report_data.create_bullet_chart('T_GRF_YT', "Trail Leg (AP axis) MAX", round(y['max']['REAR_FORCE_Y']), 50, 93, 106, 130, unit='')
    pdf.image(f'figure/bar/T_GRF_YT_bar.png', -15, 88, 235,)

    report_data.create_bullet_chart('T_GRF_ZT', "Trail Leg (Vertical) MAX", round(z['max']['REAR_FORCE_Z']), 80, 140, 162, 210, unit='')
    pdf.image(f'figure/bar/T_GRF_ZT_bar.png', -15, 101, 235,)
    
    # ==========================================================================================================
    
    pdf.set_font('helvetica', 'B', 15)
    pdf.set_y(130)
    pdf.cell(190, 7, 'Arm cocking', align = 'C', ln = True, fill = True)

    # SHOULDER EXTERNAL ROTATION

    #pdf.set_link(ch4_link, page = 3)
    pdf.set_y(145)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, '5. SHOULDER EXTERNAL ROTATION', ln = True, link = ch1_link)
    pdf.set_font('helvetica', 'B', 7)
    pdf.set_x(93)
    pdf.set_text_color(150,0,0)
    pdf.cell(0, 0, 'Click [Go to Comment]', ln = True, link = ch7_link)
    pdf.set_text_color(0,0,0)
    pdf.set_y(145)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/SHOULDER EXTERNAL ROTATION.png',x=70,y=147,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/SHOULDER EXTERNAL ROTATION.png',x=10,y=147,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/SHOULDER EXTERNAL ROTATION.png',x=10,y=147,w=50,h=50)

    data_as_dict = {
                    ""       : ["Shoulder External Rotation [°]"],
                    "FC"     : [round(ang['fc_time']['LEAD_SHOULDER_ANGLE_Z'])],
                    "MER"    : [round(ang['mer_time']['LEAD_SHOULDER_ANGLE_Z'])],
                    "BR"     : [round(ang['br_time']['LEAD_SHOULDER_ANGLE_Z'])],
                    "Max"    : [round(ang['max']['LEAD_SHOULDER_ANGLE_Z'])],
                    "Timing" : [round(100 * (ang['max_frame']['LEAD_SHOULDER_ANGLE_Z'] - k_fc_time) / (k_br_time - k_fc_time))]
                    }
    
    pdf.set_y(205)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [48,30,30,30,30,30],
                    x_start='C')

    report_data.create_bullet_chart('Shoulder External Rotation (FC)', "FC", round(ang['fc_time']['LEAD_SHOULDER_ANGLE_Z']), 0, 11, 48, 90, unit='°')
    pdf.image(f'figure/bar//Shoulder External Rotation (FC)_bar.png', -15, 220, 235,)

    report_data.create_bullet_chart('Shoulder External Rotation (MER)', "MER", round(ang['mer_time']['LEAD_SHOULDER_ANGLE_Z']), 120, 163, 178, 250, unit='°')
    pdf.image(f'figure/bar//Shoulder External Rotation (MER)_bar.png', -15, 234, 235,)

    # ==========================================================================================================

    pdf.add_page()
    
    # SHOULDER HORIZONTAL ABDUCTION

    #pdf.set_link(ch5_link, page = 4)
    pdf.set_y(12)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, '6. SHOULDER HORIZONTAL ABDUCTION', ln = True, link = ch1_link)
    pdf.set_font('helvetica', 'B', 7)
    pdf.set_x(98)
    pdf.set_text_color(150,0,0)
    pdf.cell(0, 0, 'Click [Go to Comment]', ln = True, link = ch7_link)
    pdf.set_text_color(0,0,0)
    pdf.set_y(17)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/SHOULDER HORIZONTAL ABDUCTION.png',x=70,y=14,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/SHOULDER HORIZONTAL ABDUCTION.png',x=10,y=14,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/SHOULDER HORIZONTAL ABDUCTION.png',x=10,y=14,w=50,h=50)

    data_as_dict = {
                    ""       : ["Shoulder Horizontal Abduction [°]"],
                    "FC"     : [round(ang['fc_time']['LEAD_SHOULDER_ANGLE_X'])],
                    "MER"    : [round(ang['mer_time']['LEAD_SHOULDER_ANGLE_X'])],
                    "BR"     : [round(ang['br_time']['LEAD_SHOULDER_ANGLE_X'])],
                    "Min"    : [round(ang['min']['LEAD_SHOULDER_ANGLE_X'])],
                    "Timing" : [round(100 * (ang['min_frame']['LEAD_SHOULDER_ANGLE_X'] - k_fc_time) / (k_br_time - k_fc_time))]
                    }

    pdf.set_y(64)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [48,30,30,30,30,30],
                    x_start='C')
    
    report_data.create_bullet_chart('Shoulder Horizontal Abduction (FC)', "FC", round(ang['fc_time']['LEAD_SHOULDER_ANGLE_X']), -100, -63, -24, 10, unit='°')
    pdf.image(f'figure/bar//Shoulder Horizontal Abduction (FC)_bar.png', -15, 79, 235,)

    report_data.create_bullet_chart('Shoulder Horizontal Abduction (MER)', "MER", round(ang['mer_time']['LEAD_SHOULDER_ANGLE_X']), -90, -12, 9, 30, unit='°')
    pdf.image(f'figure/bar//Shoulder Horizontal Abduction (MER)_bar.png', -15, 91, 235,)

    # ===========================================================================================================

    # LEAD LEG KNEE FLEXION

    #pdf.set_link(ch8_link, page = 4)
    pdf.set_y(108)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, '7. LEAD LEG KNEE FLEXION', ln = True, link = ch1_link)
    pdf.set_font('helvetica', 'B', 7)
    pdf.set_x(72)
    pdf.set_text_color(150,0,0)
    pdf.cell(0, 0, 'Click [Go to Comment]', ln = True, link = ch7_link)
    pdf.set_text_color(0,0,0)
    pdf.set_y(108)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/LEAD LEG KNEE FLEXION.png',x=70,y=110,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/LEAD LEG KNEE FLEXION.png',x=10,y=111,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/LEAD LEG KNEE FLEXION.png',x=10,y=111,w=50,h=50)

    data_as_dict = {
                    ""       : ["Lead Leg Knee Flexion [°]"],
                    "FC"     : [round(ang['fc_time']['LEAD_KNEE_ANGLE_X'])],
                    "MER"    : [round(ang['mer_time']['LEAD_KNEE_ANGLE_X'])],
                    "BR"     : [round(ang['br_time']['LEAD_KNEE_ANGLE_X'])],
                    "Max"    : [round(ang['max']['LEAD_KNEE_ANGLE_X'])],
                    "Timing" : [round(100 * (ang['max_frame']['LEAD_KNEE_ANGLE_X'] - k_fc_time) / (k_br_time - k_fc_time))]
                    }

    pdf.set_y(160)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [48,30,30,30,30,30],
                    x_start='C')

    report_data.create_bullet_chart('Lead Leg Knee Flexion (FC)', "FC", round(ang['fc_time']['LEAD_KNEE_ANGLE_X']), 20, 39, 66, 100, unit='°')
    pdf.image(f'figure/bar/Lead Leg Knee Flexion (FC)_bar.png', -15, 175, 235,)

    report_data.create_bullet_chart('Lead Leg Knee Flexion (BR)', "BR", round(ang['br_time']['LEAD_KNEE_ANGLE_X']), 0, 25, 53, 90, unit='°')
    pdf.image(f'figure/bar/Lead Leg Knee Flexion (BR)_bar.png', -15, 187, 235,)

    # ===========================================================================================================

    # GRF Y

    #pdf.set_link(ch11_link, page = 4)
    pdf.set_y(205)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, '8-1. LEAD LEG GROUND REACTION FORCE (AP axis)', ln = True, link = ch1_link)
    pdf.set_font('helvetica', 'B', 7)
    pdf.set_x(121)
    pdf.set_text_color(150,0,0)
    pdf.cell(0, 0, 'Click [Go to Comment]', ln = True, link = ch7_link)
    pdf.set_text_color(0,0,0)
    pdf.set_y(205)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/grf_ap.png',x=70,y=210,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/GRF_Y.png',x=17,y=210,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/GRF_Y.png',x=17,y=210,w=50,h=50)
    
    data_as_dict = {
                    ""       : ["Lead Leg [% BW]"],
                    "FC"     : [round(y['fc_time']['LEAD_FORCE_Y'])],
                    "MER"    : [round(y['mer_time']['LEAD_FORCE_Y'])],
                    "BR"     : [round(y['br_time']['LEAD_FORCE_Y'])],
                    "Min"    : [round(y['min']['LEAD_FORCE_Y'])],
                    "Timing" : [round(100 * (y['min_frame']['LEAD_FORCE_Y'] - f_fc_time) / (f_br_time - f_fc_time))]
                    }
    
    pdf.set_y(263)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [48,30,30,30,30,30],
                    x_start='C')

    # ===========================================================================================================

    pdf.add_page()

    # GRF Z

    #pdf.set_link(ch12_link, page = 5)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, '8-2. LEAD LEG  GROUND REACTION FORCE (Vertical)', ln = False, link = ch1_link)
    pdf.set_font('helvetica', 'B', 7)
    pdf.set_x(123)
    pdf.set_text_color(150,0,0)
    pdf.cell(0, 0, 'Click [Go to Comment]', ln = True, link = ch7_link)
    pdf.set_text_color(0,0,0)
    pdf.set_y(20)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/grf_vt.png',x=70,y=20,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/GRF_Z.png',x=16,y=20,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/GRF_Z.png',x=16,y=20,w=50,h=50)

    data_as_dict = {
                    ""       : ["Lead Leg [% BW]"],
                    "FC"     : [round(z['fc_time']['LEAD_FORCE_Z'])],
                    "MER"    : [round(z['mer_time']['LEAD_FORCE_Z'])],
                    "BR"     : [round(z['br_time']['LEAD_FORCE_Z'])],
                    "Max"    : [round(z['max']['LEAD_FORCE_Z'])],
                    "Timing" : [round(100 * (z['max_frame']['LEAD_FORCE_Z'] - f_fc_time) / (f_br_time - f_fc_time))]
                    }
    
    pdf.set_y(74)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [48,30,30,30,30,30],
                    x_start='C')

    pdf.set_y(16)
    # pdf.set_x(125)
    pdf.set_text_color(150,0,0)
    pdf.set_font('helvetica', 'B', 9)
    pdf.cell(5, 7, f'Blue Vertical Line ~ Red Vertical Line Time : {force_peak_time} s', ln = 1, align = '')
    pdf.set_text_color(0,0,0)

    report_data.create_bullet_chart('L_GRF_YT', "Lead Leg (AP axis) MAX", round(y['min']['LEAD_FORCE_Y']), -130, -106, -90, -50, unit='')
    pdf.image(f'figure/bar/L_GRF_YT_bar.png', -15, 89, 235,)

    report_data.create_bullet_chart('L_GRF_ZT', "Lead Leg (Vertical) MAX", round(z['max']['LEAD_FORCE_Z']), 80, 138, 162, 210, unit='')
    pdf.image(f'figure/bar/L_GRF_ZT_bar.png', -15, 103, 235,)

    # ===========================================================================================================

    pdf.set_font('helvetica', 'B', 15)
    pdf.set_y(140)
    pdf.cell(190, 7, 'Arm acceleration', align = 'C', ln = True, fill = True)

    # SHOULDER EXTERNAL ROTATION

    #pdf.set_link(ch4_link, page = 5)
    pdf.set_y(155)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, '9. SHOULDER ABDUCTION', ln = False, link = ch1_link)
    pdf.set_font('helvetica', 'B', 7)
    pdf.set_x(68)
    pdf.set_text_color(150,0,0)
    pdf.cell(0, 0, 'Click [Go to Comment]', ln = True, link = ch7_link)
    pdf.set_text_color(0,0,0)
    
    pdf.image(f'figure/SHOULDER ABDUCTION.png',x=70,y=157,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/SHOULDER ABDUCTION.png',x=10,y=158,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/SHOULDER ABDUCTION.png',x=10,y=158,w=50,h=50)

    data_as_dict = {
                    ""       : ["Shoulder Abduction [°]"],
                    "FC"     : [round(ang['fc_time']['LEAD_SHOULDER_ANGLE_Y'])],
                    "MER"    : [round(ang['mer_time']['LEAD_SHOULDER_ANGLE_Y'])],
                    "BR"     : [round(ang['br_time']['LEAD_SHOULDER_ANGLE_Y'])],
                    "Max"    : [round(ang['max']['LEAD_SHOULDER_ANGLE_Y'])],
                    "Timing" : [round(100 * (ang['max_frame']['LEAD_SHOULDER_ANGLE_Y'] - k_fc_time) / (k_br_time - k_fc_time))]
                    }

    
    pdf.set_y(210)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [48,30,30,30,30,30],
                    x_start='C')

    report_data.create_bullet_chart('Shoulder Abduction (FC)', "FC", round(ang['fc_time']['LEAD_SHOULDER_ANGLE_Y']), 40, 74, 97, 120, unit='°')
    pdf.image(f'figure/bar//Shoulder Abduction (FC)_bar.png', -15, 226, 235,)

    report_data.create_bullet_chart('Shoulder Abduction (BR)', "BR", round(ang['br_time']['LEAD_SHOULDER_ANGLE_Y']), 50, 81, 99, 120, unit='°')
    pdf.image(f'figure/bar//Shoulder Abduction (BR)_bar.png', -15, 238, 235,)

    # ===========================================================================================================
    
    pdf.add_page()
    
    # TRUNK FORWARD TILT

    #pdf.set_link(ch9_link, page = 6)
    pdf.set_y(12)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, '10. TRUNK FORWARD TILT', ln = True, link = ch1_link)
    pdf.set_font('helvetica', 'B', 7)
    pdf.set_x(68)
    pdf.set_text_color(150,0,0)
    pdf.cell(0, 0, 'Click [Go to Comment]', ln = True, link = ch7_link)
    pdf.set_text_color(0,0,0)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/TRUNK FORWARD TILT.png',x=70,y=14,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/TRUNK FORWARD TILT.png',x=10,y=14,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/TRUNK FORWARD TILT.png',x=10,y=14,w=50,h=50)
    
    data_as_dict = {
                    ""       : ["Trunk Forward Tilt [°]"],
                    "FC"     : [round(ang['fc_time']['TORSO_ANGLE_X'])],
                    "MER"    : [round(ang['mer_time']['TORSO_ANGLE_X'])],
                    "BR"     : [round(ang['br_time']['TORSO_ANGLE_X'])],
                    "Max"    : [round(ang['max']['TORSO_ANGLE_X'])],
                    "Timing" : [round(100 * (ang['max_frame']['TORSO_ANGLE_X'] - k_fc_time) / (k_br_time - k_fc_time))]
                    }

    pdf.set_y(65)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [48,30,30,30,30,30],
                    x_start='C')
    
    report_data.create_bullet_chart('Trunk Forward Tilt (BR)', "BR", round(ang['br_time']['TORSO_ANGLE_X']), 0, 22, 43, 100, unit='°')
    pdf.image(f'figure/bar/Trunk Forward Tilt (BR)_bar.png', -15, 82, 235,)

    # ===========================================================================================================

    # TRUNK LATERAL TILT

    #pdf.set_link(ch10_link, page = 6)
    pdf.set_y(104)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, '11. TRUNK LATERAL TILT', ln = False, link = ch1_link)
    pdf.set_font('helvetica', 'B', 7)
    pdf.set_x(66)
    pdf.set_text_color(150,0,0)
    pdf.cell(0, 0, 'Click [Go to Comment]', ln = True, link = ch7_link)
    pdf.set_text_color(0,0,0)

    pdf.image(f'figure/TRUNK LATERAL TILT.png',x=70,y=107,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/TRUNK LATERAL TILT.png',x=10,y=107,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/TRUNK LATERAL TILT.png',x=10,y=107,w=50,h=50)

    data_as_dict = {
                    ""       : ["Trunk Lateral Tilt [°]"],
                    "FC"     : [round(ang['fc_time']['TORSO_ANGLE_Y'])],
                    "MER"    : [round(ang['mer_time']['TORSO_ANGLE_Y'])],
                    "BR"     : [round(ang['br_time']['TORSO_ANGLE_Y'])],
                    "Max"    : [round(ang['max']['TORSO_ANGLE_Y'])],
                    "Timing" : [round(100 * (ang['max_frame']['TORSO_ANGLE_Y'] - k_fc_time) / (k_br_time - k_fc_time))]
                    }

    pdf.set_y(157)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [48,30,30,30,30,30],
                    x_start='C')
    
    report_data.create_bullet_chart('Trunk Lateral Tilt (BR)', "BR", round(ang['br_time']['TORSO_ANGLE_Y']), -20, 9, 23, 50, unit='°')
    pdf.image(f'figure/bar/Trunk Lateral Tilt (BR)_bar.png', -15, 172, 235,)
    
    # ===========================================================================================================

    # LEAD LEG KNEE EXTENSION ANGULAR VELOCITY

    #pdf.set_link(ch7_link, page = 6)
    pdf.set_y(196)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, '12. LEAD LEG KNEE EXTENSION ANGULAR VELOCITY', ln = False, link = ch1_link)
    pdf.set_font('helvetica', 'B', 7)
    pdf.set_x(123)
    pdf.set_text_color(150,0,0)
    pdf.cell(0, 0, 'Click [Go to Comment]', ln = True, link = ch7_link)
    pdf.set_text_color(0,0,0)
    pdf.set_y(196)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/LEAD LEG KNEE EXTENSION ANGULAR VELOCITY.png',x=70,y=200,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/LEAD LEG KNEE EXTENSION ANGULAR VELOCITY.png',x=10,y=200,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/LEAD LEG KNEE EXTENSION ANGULAR VELOCITY.png',x=10,y=200,w=50,h=50)

    data_as_dict = {
                    ""       : ["Lead Knee Ext. Vel.[°/s]"],
                    "FC"     : [round(ang['fc_time']['LEAD_KNEE_ANGULAR_VELOCITY_X'])],
                    "MER"    : [round(ang['mer_time']['LEAD_KNEE_ANGULAR_VELOCITY_X'])],
                    "BR"     : [round(ang['br_time']['LEAD_KNEE_ANGULAR_VELOCITY_X'])],
                    "Max"    : [round(ang['max']['LEAD_KNEE_ANGULAR_VELOCITY_X'])],
                    "Timing" : [round(100 * (ang['max_frame']['LEAD_KNEE_ANGULAR_VELOCITY_X'] - k_fc_time) / (k_br_time - k_fc_time))]
                    }

    pdf.set_y(252)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [48,30,30,30,30,30],
                    x_start='C')

    report_data.create_bullet_chart('Lead Leg Knee Extension Angular Velocity (MAX)', "MAX", round(ang['max']['LEAD_KNEE_ANGULAR_VELOCITY_X']), 0, 218, 502, 800, unit='°/s')
    pdf.image(f'figure/bar/Lead Leg Knee Extension Angular Velocity (MAX)_bar.png', -15, 267, 235,)
    
    # ===========================================================================================================

    # comment
    position_kinematic = comment_txt.find('- KINEMATIC SEQUENCE & STRIDE LENGTH')
    position_xfactor = comment_txt.find('- Hip/Shoulder Separation')
    position_elb_flx = comment_txt.find('- Elbow Flexion')
    position_trail_grf = comment_txt.find('- Trail Leg GRF')
    position_sho_er = comment_txt.find('- Shoulder ER')
    position_sho_had = comment_txt.find('- Shoulder Horizontal Abduction')
    position_knee_flex = comment_txt.find('- Lead Knee Flexion')
    position_lead_grf = comment_txt.find('- Lead Leg GRF')
    position_sho_add = comment_txt.find('- Shoulder Abduction')
    position_trk_fwd_tilt = comment_txt.find('- Trunk Forward Tilt')
    position_trk_lat_tilt = comment_txt.find('- Trunk Lataral Tilt')
    position_lead_knee_ext_vel = comment_txt.find('- Lead Leg Knee Extention Velocity')
    
    comment_kinematic = comment_txt[position_kinematic+22:position_xfactor]
    comment_xfactor = comment_txt[position_xfactor+26:position_elb_flx]
    comment_elb_flx = comment_txt[position_elb_flx+16:position_trail_grf]
    comment_trail_grf = comment_txt[position_trail_grf+15:position_sho_er]
    comment_sho_er = comment_txt[position_sho_er+13:position_sho_had]
    comment_sho_had = comment_txt[position_sho_had+31:position_knee_flex]
    comment_knee_flex = comment_txt[position_knee_flex+19:position_lead_grf]
    comment_lead_grf = comment_txt[position_lead_grf+14:position_sho_add]
    comment_sho_add = comment_txt[position_sho_add+21:position_trk_fwd_tilt]
    comment_trk_fwd_tilt = comment_txt[position_trk_fwd_tilt+20:position_trk_lat_tilt]
    comment_trk_lat_tilt = comment_txt[position_trk_lat_tilt+20:position_lead_knee_ext_vel]
    comment_lead_knee_ext_vel = comment_txt[position_lead_knee_ext_vel+34:]
    
    pdf.add_page()

    pdf.set_font('helvetica', 'B', 12)
    pdf.set_y(0)
    pdf.cell(0, 7, 'COMMENT', align = 'C', ln = True, link = ch1_link, fill = True)
    pdf.set_font('kor', 'B', 8)
    pdf.ln(3)
    pdf.cell(0, 0, '- 1. Kinematic Sequence [click : 피드백으로 복귀]', ln = 1, link = ch1_link)
    pdf.ln(4)
    pdf.multi_cell(0, 4, comment_kinematic, border = True,ln = True)
    pdf.ln(4)
    pdf.cell(0, 0, '- 2. Hip/Shoulder Separation [click : 피드백으로 복귀]', ln = True, link = ch2_link)
    pdf.ln(2)
    pdf.multi_cell(0, 4, comment_xfactor, border = True, ln = False)
    pdf.ln(4)
    pdf.cell(0, 0, '- 3. Elbow  Flexion [click : 피드백으로 복귀]', ln = False, link = ch2_link)
    pdf.ln(2)
    pdf.multi_cell(0, 4, comment_elb_flx, border = True,ln = False)
    pdf.ln(4)
    pdf.cell(0, 0, '- 4. Trail Leg Ground Reaction Force [click : 피드백으로 복귀]', ln = True, link = ch2_link)
    pdf.ln(2)
    pdf.multi_cell(0, 4, comment_trail_grf, border = True, ln = False)
    pdf.ln(4)
    pdf.cell(0, 0, '- 5. Shoulder External Roataion [click : 피드백으로 복귀]', ln = True, link = ch3_link)
    pdf.ln(2)
    pdf.multi_cell(0, 4, comment_sho_er, border = True, ln = False)
    pdf.ln(4)
    pdf.cell(0, 0, '- 6. Shoulder Horizontal Abudction [click : 피드백으로 복귀]', ln = True, link = ch4_link)
    pdf.ln(2)
    pdf.multi_cell(0, 4, comment_sho_had, border = True, ln = False)
    pdf.ln(3)
    pdf.cell(0, 0, '- 7. Lead Leg Knee Flexion [click : 피드백으로 복귀]', ln = True, link = ch4_link)
    pdf.ln(2)
    pdf.multi_cell(0, 4, comment_knee_flex, border = True, ln = False)
    pdf.ln(4)
    pdf.cell(0, 0, '- 8. Lead Leg Ground Reaction Force [click : 피드백으로 복귀]', ln = True, link = ch4_link)
    pdf.ln(2)
    pdf.multi_cell(0, 4, comment_lead_grf, border = True, ln = False) 
    pdf.ln(4)
    pdf.cell(0, 0, '- 9. Shoulder Abduction [click : 피드백으로 복귀]', ln = False, link = ch5_link)
    pdf.ln(2)
    pdf.multi_cell(0, 4, comment_sho_add, border = True, ln = False)  
    pdf.ln(4)
    pdf.cell(0, 0, '- 10. Trunk Forward Tilt [click : 피드백으로 복귀]', ln = True, link = ch6_link)
    pdf.ln(2)
    pdf.multi_cell(0, 4, comment_trk_fwd_tilt, border = True, ln = False)     
    pdf.ln(4)
    pdf.cell(0, 0, '- 11. Trunk Latral Tilt [click : 피드백으로 복귀]', ln = True, link = ch6_link)
    pdf.ln(2)
    pdf.multi_cell(0, 4, comment_trk_lat_tilt, border = True, ln = False)    
    pdf.ln(4)
    pdf.cell(0, 0, '- 12. Lead Leg Knee Extension Velocity [click : 피드백으로 복귀]', ln = True, link = ch6_link)
    pdf.ln(2)
    pdf.multi_cell(0, 4, comment_lead_knee_ext_vel, border = True, ln = False)
          
    '''
    # Summary

    pdf.set_font('helvetica', 'B', 8)
    pdf.set_y(140)
    # Add Summary Title
    pdf.set_font('helvetica', 'B', 15)
    pdf.cell(0, 10, 'Summary', ln=True, align='C')
    
    # Kinematic Sequence
    pdf.set_font('helvetica', 'B', 15)
    pdf.set_y(155)
    pdf.cell(190, 10, 'Kinematic Sequence', align = 'C', ln = True, fill = True)

    pro_dic = {
        'Pelvic Angular Velocity(MAX)': '649 ~ 840°/s',
        'Torso Angular Velocity(MAX)': '987 ~ 1174°/s',
        'Elbow Angular Velocity(MAX)': '2211 ~ 2710°/s',
        'Shoulder Angular Velocity(MAX)': '4331 ~ 4884°/s'
    }
    
    mine_val = [peak_pel, peak_tor, peak_elb, peak_sho]

    pdf.set_font('helvetica', 'B', 8)
    data_as_dict = {"" : list(pro_dic.keys()),
                    "My Data" : mine_val,
                    "Pro" : list(pro_dic.values())}
    pdf.set_y(170)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [90, 45, 45],
                    x_start = 15)
    
    # GRF
    pdf.set_font('helvetica', 'B', 15)
    pdf.set_y(215)
    pdf.cell(190, 10, 'GROUND REACTION FORCE', align = 'C', ln = True, fill = True)

    pro_dic = {
        'Trail Leg Y': '80 ~ 106',
        'Trail Leg Z': '128 ~ 162',
        'Lead Leg Y': '113 ~ 152',
        'Lead Leg Z': '194 ~ 260'
    }

    mine_val = [
                max_rear_y, max_rear_z,
                max_lead_y, max_lead_z
    ]

    pdf.set_font('helvetica', 'B', 8)
    data_as_dict = {"" : list(pro_dic.keys()),
                    "My Data" : mine_val,
                    "Pro" : list(pro_dic.values())}
    pdf.set_y(230)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [90, 45, 45],
                    x_start = 15)
    
    # ===========================================================================================================
    pdf.add_page()
    pdf.set_y(10)
    # Add Summary Title
    pdf.set_font('helvetica', 'B', 15)
    pdf.cell(0, 10, 'Summary', ln=True, align='C')
    
    
    # Foot Contact
    pdf.set_font('helvetica', 'B', 15)
    pdf.set_y(30)
    pdf.cell(190, 10, 'Foot Contact', align = 'C', ln = True, fill = True)

    pro_dic = {
        'Elbow Flexion': '76 ~ 115°',
        'Shoulder External Rotation': '11 ~ 48°',
        'Shoulder Abduction': '74 ~ 97°',
        'Shoulder Horizontal Abduction': '-63 ~ -24°',
        'Trunk Forward Tilt': '-12 ~ 9°',
        'Lead Leg Knee Flexion': '39 ~ 66°',
    }
    
    mine_val = [
                EF_fp, SER_fc, SAB_fp,
                SHA_fp, TFT_fp, LKF_fp
    ]

    pdf.set_font('helvetica', 'B', 8)
    data_as_dict = {"" : list(pro_dic.keys()),
                    "My Data" : mine_val,
                    "Pro" : list(pro_dic.values())}
    pdf.set_y(45)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [90, 45, 45],
                    x_start = 15)
    
    # Arm Cocking
    pdf.set_font('helvetica', 'B', 15)
    pdf.set_y(110)
    pdf.cell(190, 10, 'Arm Cocking', align = 'C', ln = True, fill = True)

    pro_dic = {
        'Elbow Flexion': '82 ~ 98°',
        'Shoulder Horizontal Abduction': '-12 ~ 9°',
        'Shoulder External Rotation': '163 ~ 178°'
    }

    mine_val = [
                EF_mer, SHA_mer, SER_mer
    ]

    pdf.set_font('helvetica', 'B', 15)
    data_as_dict = {"" : list(pro_dic.keys()),
                    "My Data" : mine_val,
                    "Pro" : list(pro_dic.values())}
    pdf.set_y(125)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [90, 45, 45],
                    x_start = 15)
 
    # Ball Release
    pdf.set_font('helvetica', 'B', 15)
    pdf.set_y(200)
    pdf.cell(190, 10, 'Ball Release', align = 'C', ln = True, fill = True)

    pro_dic = {
        'Lead Leg Knee Flexion': '25 ~ 53°',
        'Elbow Flexion': '27 ~ 28°',
        'Shoulder Abduction': '81 ~ 99°',
        'Trunk Forward Tilt': '22 ~ 43°',
        'Trunk Lateral Tilt': '9 ~ 23°'
    }

    mine_val = [
                LKF_br, EF_br, SAB_br,
                TFT_br, TLT_br
    ]
        

    pdf.set_font('helvetica', 'B', 8)
    data_as_dict = {"" : list(pro_dic.keys()),
                    "My Data" : mine_val,
                    "Pro" : list(pro_dic.values())}
    pdf.set_y(215)
    pdf.create_table(table_data = data_as_dict,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [90, 45, 45],
                    x_start = 15)
    '''

    figure_file = glob('figure/*png')
    for figure in figure_file:
        os.remove(figure)
        
    pdf.output(f'{store}/{day}_{player}_{trial}_{side}_baseball_report.pdf')

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:07<00:00,  7.88s/it]
